#### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1sCb1YbQ3-1oiL-cnK0yCkwFcQpvpf0efIngAUeC1ixo"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

### NY State polio vax rates by county

In [10]:
html = urlopen(
    "https://health.ny.gov/diseases/communicable/polio/county_vaccination_rates.htm"
)

In [11]:
soup = BeautifulSoup(html, "html.parser")

In [12]:
table = soup.find_all("table")[0]

In [13]:
headers = (table.find_all("tr")[0]).find_all("th")

In [14]:
colnames = []
for header in headers:
    colnames.append(header.text.strip())

In [15]:
rows = table.find_all("tr")

In [16]:
county = []
number_todate = []
popsize = []
polio_vaxrate = []
for row in rows[1:64]:
    cells = row.find_all("td")
    counties = row.find_all("th")
    if (len(counties) > 0) & (len(cells) > 2):
        county.append(counties[0].text.strip())
        number_todate.append(cells[0].text.strip())
        popsize.append(cells[1].text.strip())
        polio_vaxrate.append(cells[2].text.strip())

In [17]:
df = pd.DataFrame(
    {
        str(colnames[0]): county,
        str(colnames[1]): number_todate,
        str(colnames[2]): popsize,
        str(colnames[3]): polio_vaxrate,
        "agegroup": "2-year-olds",
    }
)
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [18]:
df

,county,number_up_to_date,population_size,polio_vaccination_rate,agegroup
0,Albany,"2,705","3,198",84.58%,2-year-olds
1,Allegany,380,492,77.24%,2-year-olds
2,Broome,"1,721","2,026",84.95%,2-year-olds
3,Cattaraugus,666,857,77.71%,2-year-olds
4,Cayuga,653,762,85.70%,2-year-olds
5,Chautauqua,"1,148","1,354",84.79%,2-year-olds
6,Chemung,785,912,86.07%,2-year-olds
7,Chenango,396,476,83.19%,2-year-olds
8,Clinton,652,706,92.35%,2-year-olds
9,Columbia,404,540,74.81%,2-year-olds


#### manually add nyc boroughs from here:
##### https://www1.nyc.gov/assets/doh/downloads/pdf/cd/polio-vaccination-coverage-by-zip.pdf

In [19]:
boroughs = pd.DataFrame(
    {
        str(colnames[0]): ["New York", "Bronx", "Kings", "Queens", "Richmond"],
        str(colnames[1]): [0, 0, 0, 0, 0],
        str(colnames[2]): [0, 0, 0, 0, 0],
        str(colnames[3]): [91.0, 87.8, 81.2, 89.6, 81.7],
        "agegroup": "children between the ages of 6 months and 5 years",
    }
)
boroughs.columns = boroughs.columns.str.lower().str.replace(" ", "_")

In [20]:
final = pd.concat([df, boroughs]).reset_index().drop(columns={"index"})

In [21]:
dw.add_data(chart_id="A5jS1", data=final)

<Response [204]>

In [22]:
src = pd.read_csv("data/raw/polio-vaccine-coverage-of-one-year-olds.csv")

In [23]:
src = src.rename(columns={"Pol3 (% of one-year-olds immunized)": "polio_vaxrate"})

In [24]:
src.columns = src.columns.str.lower().str.replace(" ", "_")

In [25]:
countries = src[src["year"] == 2019]

In [26]:
# dw.add_data(chart_id="zXfEf", data=countries)